In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

In [3]:
# Import the FoodDemandForecaster class
from forecaster import FoodDemandForecaster

# Create an instance of the forecaster
forecaster = FoodDemandForecaster()

# Load the data (this will automatically call preprocess_data() which includes region mapping and meal filtering)
forecaster.load_data()

# If you want to view the regional mapping after it's been created
print("\nRegional Mapping:")
for center_id, region in forecaster.regional_mapping.items():
    print(f"Center ID {center_id} -> {region}")

# Check the dataset before and after filtering
# The original dataset should be in forecaster.train_data
print("\nData shape after filtering:")
print(f"Train data shape: {forecaster.train_data.shape}")
print(f"Unique meal IDs remaining: {len(forecaster.train_data['meal_id'].unique())}")

# You can also access the filtered meal IDs that were removed
removed_meals = [1571, 2104, 2956, 2490, 2569, 2664]
print(f"\nMeals filtered out: {removed_meals}")

# To see data availability analysis results
print("\nData Availability Analysis:")
for key, info in list(forecaster.data_availability.items())[:5]:  # Show first 5 items
    print(f"{key}: {info['records']} records, {info['weeks']} weeks, Sufficient: {info['sufficient']}")

c:\python_projects\prophet-demand-forecast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


🍽️ Loaded 45 meal names from meal_database.json
   • 1885: Basic Vegetable Pasta Bake
   • 1993: Simple Spinach and Chickpea Curry
   • 2539: Basic Tofu Stir Fry
   • ... and 42 more
✅ Loaded train.csv: 456548 records
📋 Train columns: ['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price', 'emailer_for_promotion', 'homepage_featured', 'num_orders']
ℹ️ test.csv not found - continuing without it (not required for forecasting)
🍽️ Filtering meals with insufficient coverage...
📊 Initial meals: 51 total
📊 Initial records: 456,548
🗑️ Removing meals: [1571, 2104, 2956, 2490, 2569, 2664]
✅ Filtering complete:
   • Meals removed: 6
   • Records removed: 38,746
   • Remaining meals: 45
   • Remaining records: 417,802
   • Data reduction: 8.5%
📍 Loading regional mapping from fulfilment_center_info.csv
📋 Fulfilment center data loaded: 77 centers
📋 Columns: ['center_id', 'city_code', 'region_code', 'center_type', 'op_area']
🗺️ Regional mapping created from fulfilment center info:
   R

KeyError: 'records'

In [ ]:
# remove Lyon
forecaster.train_data = forecaster.train_data.loc[forecaster.train_data['region'] != "Region-Lyon"]

In [30]:
forecaster.train_data.drop(columns=['emailer_for_promotion', 'homepage_featured', 'base_price'], inplace=True)

In [16]:
print("Data Points in Total by region:")
forecaster.train_data['region'].value_counts()

Data Points in Total by region:


region
Region-Kassel    176117
Region-Luzern    105864
Region-Wien       86302
Name: count, dtype: int64

In [12]:
# Average per Region
regions = ['Region-Kassel', "Region-Luzern", "Region-Wien"]
for region in regions:
    len(forecaster.train_data[forecaster.train_data['region'] == region]) / 145
    print(f"Average data points per week in {region}: {len(forecaster.train_data[forecaster.train_data['region'] == region]) / 145:.2f}")

Average data points per week in Region-Kassel: 1214.60
Average data points per week in Region-Luzern: 730.10
Average data points per week in Region-Wien: 595.19


In [15]:
for region in regions:
    print(f"Average number of meals per week in {region}: {forecaster.train_data['num_orders'][forecaster.train_data['region'] == region].sum() / 145:.2f}")

Average number of meals per week in Region-Kassel: 381516.63
Average number of meals per week in Region-Luzern: 146010.45
Average number of meals per week in Region-Wien: 124915.57


In [29]:
for region in regions:
    region_data = forecaster.train_data[forecaster.train_data['region'] == region]
    # Calculate revenue per record
    region_data['revenue'] = region_data['num_orders'] * region_data['checkout_price']
    
    # Group by week and sum revenue
    weekly_revenue = region_data.groupby('week')['revenue'].sum().reset_index()
    
    # Calculate statistics
    total_weeks = weekly_revenue['week'].nunique()
    total_revenue = weekly_revenue['revenue'].sum()
    mean_revenue = weekly_revenue['revenue'].mean()
    
    # Format numbers with thousands separator (European style)
    total_revenue_formatted = f"{total_revenue:,.2f}"
    mean_revenue_formatted = f"{mean_revenue:,.2f}"
    total_avg_formatted = f"{total_revenue/145:,.2f}"
    
    print(f"Region: {region}")
    print(f"  - Total weeks with data: {total_weeks}")
    print(f"  - Total revenue: {total_revenue_formatted} EUR")
    print(f"  - Average revenue per week: {mean_revenue_formatted} EUR")

Region: Region-Kassel
  - Total weeks with data: 145
  - Total revenue: 14,277,983,431.71 EUR
  - Average revenue per week: 98,468,851.25 EUR
Region: Region-Luzern
  - Total weeks with data: 145
  - Total revenue: 5,788,937,305.94 EUR
  - Average revenue per week: 39,923,705.56 EUR
Region: Region-Wien
  - Total weeks with data: 145
  - Total revenue: 4,847,332,717.01 EUR
  - Average revenue per week: 33,429,880.81 EUR


In [31]:
forecaster.train_data

,id,week,center_id,meal_id,checkout_price,num_orders,region
0,1379560,1,55,1885,136.83,177,Region-Kassel
1,1466964,1,55,1993,136.83,270,Region-Kassel
2,1346989,1,55,2539,134.86,189,Region-Kassel
3,1338232,1,55,2139,339.50,54,Region-Kassel
4,1448490,1,55,2631,243.50,40,Region-Kassel
...,...,...,...,...,...,...,...
456540,1035758,145,61,1525,319.13,134,Region-Wien
456541,1010438,145,61,2704,321.13,67,Region-Wien
456542,1116711,145,61,2492,455.93,42,Region-Wien
456543,1271326,145,61,1543,484.09,68,Region-Wien


In [34]:
import pandas as pd
import json

# Load the meal database from JSON file
with open('meal_database.json', 'r') as file:
    meal_data = json.load(file)

# Create a list to store the meal information
meals_list = []

# Extract data from the JSON structure
for meal_id, details in meal_data.items():
    # Calculate average ingredient weight
    avg_ingredient_weight = sum(details['kg_per_10_persons']) / len(details['kg_per_10_persons'])
    
    # Calculate perishability ratio (percentage of perishable ingredients)
    perishability = sum(details['perishable']) / len(details['perishable']) * 100
    
    # Create a dictionary for this meal
    meal_info = {
        'meal_id': int(meal_id),
        'name': details['name'],
    }
    
    meals_list.append(meal_info)

# Create DataFrame
meals_df = pd.DataFrame(meals_list)

# Sort by meal_id for better readability
meals_df = meals_df.sort_values('meal_id').reset_index(drop=True)

# Display the DataFrame
print(f"Loaded {len(meals_df)} meals from the database")
meals_df.head()

Loaded 45 meals from the database


,meal_id,name
0,1062,Beef Goulash
1,1109,Wild Mushroom Risotto
2,1198,Vegetable Stir Fry
3,1207,Authentic Carbonara
4,1216,Gourmet Curry Selection


In [ ]:
import json

# Load the meal database from JSON file
with open('meal_database.json', 'r') as file:
    meal_data = json.load(file)

# Create a list to store the ingredients information
ingredients_list = []

# Extract ingredient data from each meal
for meal_id, details in meal_data.items():
    for i, ingredient in enumerate(details['ingredients']):
        # Get the corresponding data for this ingredient
        kg_per_10 = details['kg_per_10_persons'][i]
        price_pct = details['price_distribution'][i]
        is_perishable = details['perishable'][i]
        
        # Create a record for this ingredient
        ingredient_info = {
            'meal_id': int(meal_id),
            'meal_name': details['name'],
            'ingredient': ingredient,
            'kg_per_10_persons': kg_per_10,
            'price_percentage': price_pct,
            'is_perishable': is_perishable,
        }
        
        ingredients_list.append(ingredient_info)

# Create DataFrame
ingredients_df = pd.DataFrame(ingredients_list)

# Display information about the DataFrame
print(f"Created ingredients DataFrame with {len(ingredients_df)} records")
print(f"Number of unique ingredients: {ingredients_df['ingredient'].nunique()}")
print(f"Number of meals: {ingredients_df['meal_id'].nunique()}")
ingredients_df.head()

Created ingredients DataFrame with 225 records
Number of unique ingredients: 202
Number of meals: 45


,meal_id,meal_name,ingredient,kg_per_10_persons,price_percentage,is_perishable
0,1885,Basic Vegetable Pasta Bake,Pasta,2.00,30,False
1,1885,Basic Vegetable Pasta Bake,Fresh mixed vegetables,1.50,40,True
2,1885,Basic Vegetable Pasta Bake,Powdered cheese sauce mix,0.20,20,False
3,1885,Basic Vegetable Pasta Bake,Dried herb blend,0.05,5,False
4,1885,Basic Vegetable Pasta Bake,Breadcrumb topping,0.20,5,False


In [ ]:
{
  "1885": {
    "name": "Basic Vegetable Pasta Bake",
    "ingredients": [
      "Pasta",
      "Fresh mixed vegetables",
      "Powdered cheese sauce mix",
      "Dried herb blend",
      "Breadcrumb topping"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.2,
      0.05,
      0.2
    ],
    "price_distribution": [
      30,
      40,
      20,
      5,
      5
    ],
    "perishable": [
      false,
      true,
      false,
      false,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Budget"
  },
  "1993": {
    "name": "Simple Spinach and Chickpea Curry",
    "ingredients": [
      "Dried chickpeas",
      "Fresh spinach",
      "Curry powder mix",
      "White rice",
      "Flatbread"
    ],
    "kg_per_10_persons": [
      1.0,
      1.5,
      0.1,
      2.0,
      0.5
    ],
    "price_distribution": [
      25,
      35,
      15,
      15,
      10
    ],
    "perishable": [
      false,
      true,
      false,
      false,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Budget"
  },
  "2539": {
    "name": "Basic Tofu Stir Fry",
    "ingredients": [
      "Plain tofu",
      "Fresh vegetable mix",
      "Soy sauce",
      "White rice",
      "Vegetable oil"
    ],
    "kg_per_10_persons": [
      1.5,
      1.5,
      0.3,
      2.0,
      0.2
    ],
    "price_distribution": [
      45,
      35,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      false,
      false,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Budget"
  },
  "1248": {
    "name": "Simple Swedish Meatballs",
    "ingredients": [
      "Beef/pork meatballs",
      "Gravy mix",
      "Lingonberry jam",
      "Mashed potatoes",
      "Fresh vegetables"
    ],
    "kg_per_10_persons": [
      1.5,
      0.3,
      0.3,
      2.0,
      1.0
    ],
    "price_distribution": [
      60,
      5,
      10,
      15,
      10
    ],
    "perishable": [
      true,
      false,
      false,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Budget"
  },
  "2631": {
    "name": "Basic Grilled Fish",
    "ingredients": [
      "White fish fillet",
      "Lemon-herb seasoning",
      "Potato wedges",
      "Fresh vegetables",
      "Garlic butter"
    ],
    "kg_per_10_persons": [
      2.0,
      0.1,
      2.0,
      1.0,
      0.2
    ],
    "price_distribution": [
      65,
      5,
      10,
      15,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Budget"
  },
  "1311": {
    "name": "Fish Tacos",
    "ingredients": [
      "White fish pieces",
      "Corn tortillas",
      "Cabbage slaw",
      "Lime crema",
      "Basic salsa"
    ],
    "kg_per_10_persons": [
      1.5,
      0.8,
      1.0,
      0.5,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      10,
      10
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Standard"
  },
  "1062": {
    "name": "Beef Goulash",
    "ingredients": [
      "Beef chunks",
      "Paprika sauce",
      "Potatoes",
      "Bell peppers",
      "Sour cream"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      0.8,
      0.5
    ],
    "price_distribution": [
      65,
      10,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Standard"
  },
  "1778": {
    "name": "Chicken Curry",
    "ingredients": [
      "Chicken pieces",
      "Curry sauce",
      "Basmati rice",
      "Naan bread",
      "Fresh herbs"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      0.7,
      0.05
    ],
    "price_distribution": [
      55,
      15,
      15,
      12,
      3
    ],
    "perishable": [
      true,
      true,
      false,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Standard"
  },
  "1803": {
    "name": "Vegetable Grain Bowl",
    "ingredients": [
      "Roasted vegetables",
      "Mixed grains",
      "Hummus",
      "Herb dressing",
      "Seeds"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.5,
      0.3,
      0.1
    ],
    "price_distribution": [
      40,
      25,
      20,
      10,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Standard"
  },
  "1198": {
    "name": "Vegetable Stir Fry",
    "ingredients": [
      "Fresh vegetables",
      "Tofu",
      "Stir-fry sauce",
      "Noodles",
      "Sesame oil"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      0.5,
      1.5,
      0.1
    ],
    "price_distribution": [
      45,
      30,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Standard"
  },
  "2707": {
    "name": "Seafood Rice",
    "ingredients": [
      "Mixed seafood",
      "Seasoned rice",
      "Saffron",
      "Mixed vegetables",
      "Lemon wedges"
    ],
    "kg_per_10_persons": [
      1.5,
      2.0,
      0.005,
      1.0,
      0.3
    ],
    "price_distribution": [
      60,
      10,
      15,
      12,
      3
    ],
    "perishable": [
      true,
      false,
      false,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Premium"
  },
  "1847": {
    "name": "Chili Con Carne",
    "ingredients": [
      "Ground beef",
      "Bean mix",
      "Chili sauce",
      "Mexican rice",
      "Cheese and sour cream"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      0.5,
      2.0,
      0.5
    ],
    "price_distribution": [
      55,
      10,
      15,
      10,
      10
    ],
    "perishable": [
      true,
      false,
      true,
      false,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium"
  },
  "1438": {
    "name": "Beef Burgers",
    "ingredients": [
      "Beef patties",
      "Sesame buns",
      "Lettuce & tomato",
      "Cheese slices",
      "Special sauce"
    ],
    "kg_per_10_persons": [
      1.8,
      1.0,
      0.8,
      0.5,
      0.3
    ],
    "price_distribution": [
      65,
      10,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium"
  },
  "2494": {
    "name": "Roast Chicken Dinner",
    "ingredients": [
      "Roast chicken",
      "Herb potatoes",
      "Seasonal vegetables",
      "Gravy",
      "Stuffing"
    ],
    "kg_per_10_persons": [
      3.0,
      2.0,
      1.5,
      0.5,
      0.5
    ],
    "price_distribution": [
      60,
      15,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium"
  },
  "2760": {
    "name": "Chicken Schnitzel",
    "ingredients": [
      "Breaded chicken breast",
      "Potato salad",
      "Mixed greens",
      "Lemon wedge",
      "Herb sauce"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      0.3,
      0.5
    ],
    "price_distribution": [
      60,
      15,
      10,
      5,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium"
  },
  "1109": {
    "name": "Wild Mushroom Risotto",
    "ingredients": [
      "Arborio rice",
      "Mixed mushrooms",
      "White wine reduction",
      "Parmesan",
      "Truffle oil"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.3,
      0.5,
      0.05
    ],
    "price_distribution": [
      20,
      35,
      15,
      15,
      15
    ],
    "perishable": [
      false,
      true,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Executive"
  },
  "2290": {
    "name": "Beef Stroganoff",
    "ingredients": [
      "Tender beef strips",
      "Mushroom cream sauce",
      "Egg noodles",
      "Green beans",
      "Paprika garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      1.5,
      1.0,
      0.05
    ],
    "price_distribution": [
      65,
      15,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Executive"
  },
  "1525": {
    "name": "Chicken Parmesan",
    "ingredients": [
      "Breaded chicken cutlets",
      "Marinara sauce",
      "Mozzarella",
      "Spaghetti",
      "Fresh basil"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      0.8,
      1.5,
      0.05
    ],
    "price_distribution": [
      55,
      10,
      15,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive"
  },
  "2704": {
    "name": "Gourmet Vegetable Frittata",
    "ingredients": [
      "Free-range eggs",
      "Seasonal vegetables",
      "Artisanal cheese",
      "Fresh herbs",
      "Mixed greens salad"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.5,
      0.1,
      1.0
    ],
    "price_distribution": [
      40,
      20,
      25,
      5,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Executive"
  },
  "1878": {
    "name": "Beef and Broccoli",
    "ingredients": [
      "Marinated beef strips",
      "Broccoli florets",
      "Oyster sauce",
      "Jasmine rice",
      "Toasted sesame garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.3,
      2.0,
      0.1
    ],
    "price_distribution": [
      65,
      15,
      5,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      false
    ],
    "type": "Meat",
    "price_tier": "Executive"
  },
  "2640": {
    "name": "Tuna Nicoise Salad",
    "ingredients": [
      "Seared tuna",
      "Cooked eggs",
      "Green beans",
      "Baby potatoes",
      "Dijon vinaigrette"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      1.0,
      1.5,
      0.3
    ],
    "price_distribution": [
      60,
      10,
      10,
      10,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Executive"
  },
  "2577": {
    "name": "Chicken Noodle Soup",
    "ingredients": [
      "Roasted chicken",
      "Egg noodles",
      "Root vegetables",
      "Rich broth",
      "Fresh herbs"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      1.5,
      2.0,
      0.1
    ],
    "price_distribution": [
      50,
      15,
      15,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive"
  },
  "1754": {
    "name": "BBQ Chicken Platter",
    "ingredients": [
      "BBQ chicken",
      "House sauce",
      "Coleslaw",
      "Cornbread",
      "Pickled vegetables"
    ],
    "kg_per_10_persons": [
      2.5,
      0.5,
      1.0,
      1.0,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive"
  },
  "1971": {
    "name": "Gourmet Grain Bowl",
    "ingredients": [
      "Mixed beans",
      "Ancient grains",
      "House salsa",
      "Avocado",
      "Lime crema"
    ],
    "kg_per_10_persons": [
      1.0,
      1.5,
      0.5,
      1.0,
      0.3
    ],
    "price_distribution": [
      20,
      25,
      15,
      30,
      10
    ],
    "perishable": [
      false,
      false,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Executive"
  },
  "2306": {
    "name": "Chicken and Rice Casserole",
    "ingredients": [
      "Free-range chicken",
      "Wild rice blend",
      "Creamy mushroom sauce",
      "Seasonal vegetables",
      "Herb crust"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      1.5,
      0.3
    ],
    "price_distribution": [
      55,
      15,
      15,
      10,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive"
  },
  "2826": {
    "name": "Slow-Cooked Beef Ragu",
    "ingredients": [
      "Slow-simmered beef",
      "San Marzano tomato sauce",
      "Fresh pasta",
      "Aged Parmesan",
      "Fresh basil"
    ],
    "kg_per_10_persons": [
      2.5,
      1.5,
      2.0,
      0.5,
      0.1
    ],
    "price_distribution": [
      55,
      15,
      15,
      12,
      3
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Specialty"
  },
  "2139": {
    "name": "Mediterranean Vegetable Stew",
    "ingredients": [
      "Organic vegetables",
      "Imported olive oil",
      "Mediterranean herbs",
      "Pearl couscous",
      "Feta cheese"
    ],
    "kg_per_10_persons": [
      3.0,
      0.3,
      0.1,
      2.0,
      0.5
    ],
    "price_distribution": [
      45,
      20,
      10,
      15,
      10
    ],
    "perishable": [
      true,
      false,
      false,
      false,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Specialty"
  },
  "1230": {
    "name": "Teriyaki Salmon Bowl",
    "ingredients": [
      "Sustainable salmon",
      "House teriyaki glaze",
      "Sushi rice",
      "Asian vegetables",
      "Black sesame"
    ],
    "kg_per_10_persons": [
      2.0,
      0.5,
      2.0,
      1.0,
      0.05
    ],
    "price_distribution": [
      70,
      10,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      false,
      true,
      false
    ],
    "type": "Fish",
    "price_tier": "Specialty"
  },
  "1207": {
    "name": "Authentic Carbonara",
    "ingredients": [
      "Fresh egg pasta",
      "Imported pancetta",
      "Farm-fresh eggs",
      "Pecorino Romano",
      "Cracked black pepper"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      1.0,
      0.5,
      0.02
    ],
    "price_distribution": [
      25,
      35,
      15,
      23,
      2
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Specialty"
  },
  "2322": {
    "name": "Gourmet Beef Enchiladas",
    "ingredients": [
      "Braised beef",
      "Handmade tortillas",
      "House enchilada sauce",
      "Mexican cheese blend",
      "Crema"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      1.0,
      0.8,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Specialty"
  },
  "2492": {
    "name": "Mediterranean Mezze Platter",
    "ingredients": [
      "Housemade falafel",
      "Artisanal hummus",
      "Freshly baked pita",
      "Greek salad",
      "Tzatziki sauce"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      1.0,
      2.0,
      0.8
    ],
    "price_distribution": [
      30,
      20,
      15,
      25,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant"
  },
  "1216": {
    "name": "Gourmet Curry Selection",
    "ingredients": [
      "Premium vegetables",
      "House curry blend",
      "Saffron basmati rice",
      "Garlic naan",
      "Fresh herb garnish"
    ],
    "kg_per_10_persons": [
      2.5,
      0.3,
      2.0,
      1.0,
      0.1
    ],
    "price_distribution": [
      40,
      25,
      20,
      10,
      5
    ],
    "perishable": [
      true,
      false,
      false,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant"
  },
  "1727": {
    "name": "Scampi Linguine",
    "ingredients": [
      "Jumbo wild shrimp",
      "White wine garlic sauce",
      "Handmade linguine",
      "Cherry tomatoes",
      "Fresh herbs"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      0.8,
      0.1
    ],
    "price_distribution": [
      65,
      10,
      15,
      7,
      3
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Premium Restaurant"
  },
  "1902": {
    "name": "Chef's Signature Pasta",
    "ingredients": [
      "Artisanal pasta",
      "San Marzano tomato sauce",
      "Fresh basil",
      "Imported Parmesan",
      "Extra virgin olive oil"
    ],
    "kg_per_10_persons": [
      2.5,
      1.5,
      0.2,
      0.5,
      0.2
    ],
    "price_distribution": [
      30,
      25,
      5,
      30,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant"
  },
  "1247": {
    "name": "Gourmet Chicken Alfredo",
    "ingredients": [
      "Free-range chicken breast",
      "Fresh fettuccine",
      "House-made cream sauce",
      "Organic broccoli",
      "Aged Parmesan"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      1.0,
      0.3
    ],
    "price_distribution": [
      45,
      20,
      15,
      10,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium Restaurant"
  },
  "2304": {
    "name": "Signature Pesto Pasta",
    "ingredients": [
      "Bronze-die pasta",
      "House basil pesto",
      "Heirloom tomatoes",
      "Toasted pine nuts",
      "24-month aged Parmesan"
    ],
    "kg_per_10_persons": [
      2.0,
      0.5,
      1.0,
      0.2,
      0.5
    ],
    "price_distribution": [
      25,
      20,
      15,
      15,
      25
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant"
  },
  "1543": {
    "name": "Prime Beef Tacos",
    "ingredients": [
      "Prime beef",
      "Hand-pressed tortillas",
      "Micro greens",
      "Aged cheddar",
      "House-made salsa"
    ],
    "kg_per_10_persons": [
      1.8,
      1.0,
      0.3,
      0.5,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      12,
      8
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium Restaurant"
  },
  "1770": {
    "name": "Heritage Pork Stir Fry",
    "ingredients": [
      "Heritage pork tenderloin",
      "Seasonal organic vegetables",
      "Aged soy reduction",
      "Handmade noodles",
      "Rare spices"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      0.5,
      1.5,
      0.05
    ],
    "price_distribution": [
      60,
      15,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Premium Restaurant"
  },
  "2126": {
    "name": "Deluxe Fish Curry",
    "ingredients": [
      "Wild-caught fish",
      "Coconut curry sauce",
      "Organic jasmine rice",
      "Exotic vegetables",
      "Micro herb garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      1.0,
      0.1
    ],
    "price_distribution": [
      65,
      15,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      false,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Premium Restaurant"
  },
  "1558": {
    "name": "Artisanal Vegetable Lasagna",
    "ingredients": [
      "Handcrafted pasta sheets",
      "Seasonal organic vegetables",
      "San Marzano tomato sauce",
      "Artisanal béchamel",
      "Aged cheese blend"
    ],
    "kg_per_10_persons": [
      2.0,
      2.5,
      1.0,
      1.0,
      0.8
    ],
    "price_distribution": [
      25,
      20,
      20,
      15,
      20
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining"
  },
  "2581": {
    "name": "Chef's Signature Soup",
    "ingredients": [
      "Heirloom lentils",
      "Seasonal organic vegetables",
      "House-made vegetable stock",
      "Imported herbs",
      "Artisanal sourdough"
    ],
    "kg_per_10_persons": [
      1.5,
      2.0,
      3.0,
      0.2,
      1.0
    ],
    "price_distribution": [
      25,
      30,
      15,
      15,
      15
    ],
    "perishable": [
      false,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining"
  },
  "1962": {
    "name": "Slow-Braised Chicken",
    "ingredients": [
      "Heritage chicken",
      "Organic root vegetables",
      "House-made bone broth",
      "Handcrafted dumplings",
      "Rare herb blend"
    ],
    "kg_per_10_persons": [
      3.0,
      2.0,
      2.0,
      1.0,
      0.1
    ],
    "price_distribution": [
      55,
      15,
      15,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Fine Dining"
  },
  "2444": {
    "name": "Signature Sweet & Sour",
    "ingredients": [
      "Free-range chicken",
      "House-made sweet & sour sauce",
      "Organic vegetables",
      "Fresh pineapple",
      "Premium jasmine rice"
    ],
    "kg_per_10_persons": [
      2.5,
      1.0,
      1.5,
      1.0,
      2.0
    ],
    "price_distribution": [
      60,
      10,
      15,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Fine Dining"
  },
  "2867": {
    "name": "Gourmet Stuffed Peppers",
    "ingredients": [
      "Heirloom bell peppers",
      "Ancient grain filling",
      "Imported tomato coulis",
      "Artisanal cheese",
      "Micro herb garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      0.8,
      0.1
    ],
    "price_distribution": [
      30,
      25,
      15,
      25,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining"
  },
  "1445": {
    "name": "Chef's Quiche",
    "ingredients": [
      "All-butter pastry",
      "Organic free-range eggs",
      "Seasonal vegetables",
      "Imported cheese",
      "Truffle essence"
    ],
    "kg_per_10_persons": [
      1.5,
      2.0,
      1.5,
      0.8,
      0.05
    ],
    "price_distribution": [
      20,
      25,
      15,
      25,
      15
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining"
  }
}

In [ ]:
{
  "1885": {
    "name": "Basic Vegetable Pasta Bake",
    "ingredients": [
      "Pasta",
      "Fresh mixed vegetables",
      "Powdered cheese sauce mix",
      "Dried herb blend",
      "Breadcrumb topping"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.2,
      0.05,
      0.2
    ],
    "price_distribution": [
      30,
      40,
      20,
      5,
      5
    ],
    "perishable": [
      false,
      true,
      false,
      false,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Budget",
    "storage_m3": [
      0.005,
      0.008,
      0.0005,
      0.0001,
      0.0006
    ]
  },
  "1993": {
    "name": "Simple Spinach and Chickpea Curry",
    "ingredients": [
      "Dried chickpeas",
      "Fresh spinach",
      "Curry powder mix",
      "White rice",
      "Flatbread"
    ],
    "kg_per_10_persons": [
      1.0,
      1.5,
      0.1,
      2.0,
      0.5
    ],
    "price_distribution": [
      25,
      35,
      15,
      15,
      10
    ],
    "perishable": [
      false,
      true,
      false,
      false,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Budget",
    "storage_m3": [
      0.002,
      0.009,
      0.0003,
      0.004,
      0.002
    ]
  },
  "2539": {
    "name": "Basic Tofu Stir Fry",
    "ingredients": [
      "Plain tofu",
      "Fresh vegetable mix",
      "Soy sauce",
      "White rice",
      "Vegetable oil"
    ],
    "kg_per_10_persons": [
      1.5,
      1.5,
      0.3,
      2.0,
      0.2
    ],
    "price_distribution": [
      45,
      35,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      false,
      false,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Budget",
    "storage_m3": [
      0.003,
      0.008,
      0.0003,
      0.004,
      0.0002
    ]
  },
  "1248": {
    "name": "Simple Swedish Meatballs",
    "ingredients": [
      "Beef/pork meatballs",
      "Gravy mix",
      "Lingonberry jam",
      "Mashed potatoes",
      "Fresh vegetables"
    ],
    "kg_per_10_persons": [
      1.5,
      0.3,
      0.3,
      2.0,
      1.0
    ],
    "price_distribution": [
      60,
      5,
      10,
      15,
      10
    ],
    "perishable": [
      true,
      false,
      false,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Budget",
    "storage_m3": [
      0.003,
      0.0004,
      0.0005,
      0.004,
      0.005
    ]
  },
  "2631": {
    "name": "Basic Grilled Fish",
    "ingredients": [
      "White fish fillet",
      "Lemon-herb seasoning",
      "Potato wedges",
      "Fresh vegetables",
      "Garlic butter"
    ],
    "kg_per_10_persons": [
      2.0,
      0.1,
      2.0,
      1.0,
      0.2
    ],
    "price_distribution": [
      65,
      5,
      10,
      15,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Budget",
    "storage_m3": [
      0.004,
      0.0002,
      0.005,
      0.005,
      0.0002
    ]
  },
  "1311": {
    "name": "Fish Tacos",
    "ingredients": [
      "White fish pieces",
      "Corn tortillas",
      "Cabbage slaw",
      "Lime crema",
      "Basic salsa"
    ],
    "kg_per_10_persons": [
      1.5,
      0.8,
      1.0,
      0.5,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      10,
      10
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Standard",
    "storage_m3": [
      0.003,
      0.0025,
      0.006,
      0.0007,
      0.0006
    ]
  },
  "1062": {
    "name": "Beef Goulash",
    "ingredients": [
      "Beef chunks",
      "Paprika sauce",
      "Potatoes",
      "Bell peppers",
      "Sour cream"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      0.8,
      0.5
    ],
    "price_distribution": [
      65,
      10,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Standard",
    "storage_m3": [
      0.004,
      0.0015,
      0.004,
      0.005,
      0.0007
    ]
  },
  "1778": {
    "name": "Chicken Curry",
    "ingredients": [
      "Chicken pieces",
      "Curry sauce",
      "Basmati rice",
      "Naan bread",
      "Fresh herbs"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      0.7,
      0.05
    ],
    "price_distribution": [
      55,
      15,
      15,
      12,
      3
    ],
    "perishable": [
      true,
      true,
      false,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Standard",
    "storage_m3": [
      0.004,
      0.0015,
      0.004,
      0.0025,
      0.0003
    ]
  },
  "1803": {
    "name": "Vegetable Grain Bowl",
    "ingredients": [
      "Roasted vegetables",
      "Mixed grains",
      "Hummus",
      "Herb dressing",
      "Seeds"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.5,
      0.3,
      0.1
    ],
    "price_distribution": [
      40,
      25,
      20,
      10,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Standard",
    "storage_m3": [
      0.0075,
      0.003,
      0.0007,
      0.0004,
      0.0001
    ]
  },
  "1198": {
    "name": "Vegetable Stir Fry",
    "ingredients": [
      "Fresh vegetables",
      "Tofu",
      "Stir-fry sauce",
      "Noodles",
      "Sesame oil"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      0.5,
      1.5,
      0.1
    ],
    "price_distribution": [
      45,
      30,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Standard",
    "storage_m3": [
      0.01,
      0.002,
      0.0006,
      0.003,
      0.0001
    ]
  },
  "2707": {
    "name": "Seafood Rice",
    "ingredients": [
      "Mixed seafood",
      "Seasoned rice",
      "Saffron",
      "Mixed vegetables",
      "Lemon wedges"
    ],
    "kg_per_10_persons": [
      1.5,
      2.0,
      0.005,
      1.0,
      0.3
    ],
    "price_distribution": [
      60,
      10,
      15,
      12,
      3
    ],
    "perishable": [
      true,
      false,
      false,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Premium",
    "storage_m3": [
      0.004,
      0.004,
      0.00001,
      0.005,
      0.0005
    ]
  },
  "1847": {
    "name": "Chili Con Carne",
    "ingredients": [
      "Ground beef",
      "Bean mix",
      "Chili sauce",
      "Mexican rice",
      "Cheese and sour cream"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      0.5,
      2.0,
      0.5
    ],
    "price_distribution": [
      55,
      10,
      15,
      10,
      10
    ],
    "perishable": [
      true,
      false,
      true,
      false,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium",
    "storage_m3": [
      0.003,
      0.002,
      0.0006,
      0.004,
      0.0008
    ]
  },
  "1438": {
    "name": "Beef Burgers",
    "ingredients": [
      "Beef patties",
      "Sesame buns",
      "Lettuce & tomato",
      "Cheese slices",
      "Special sauce"
    ],
    "kg_per_10_persons": [
      1.8,
      1.0,
      0.8,
      0.5,
      0.3
    ],
    "price_distribution": [
      65,
      10,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium",
    "storage_m3": [
      0.004,
      0.005,
      0.004,
      0.0007,
      0.0003
    ]
  },
  "2494": {
    "name": "Roast Chicken Dinner",
    "ingredients": [
      "Roast chicken",
      "Herb potatoes",
      "Seasonal vegetables",
      "Gravy",
      "Stuffing"
    ],
    "kg_per_10_persons": [
      3.0,
      2.0,
      1.5,
      0.5,
      0.5
    ],
    "price_distribution": [
      60,
      15,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium",
    "storage_m3": [
      0.009,
      0.004,
      0.008,
      0.0006,
      0.001
    ]
  },
  "2760": {
    "name": "Chicken Schnitzel",
    "ingredients": [
      "Breaded chicken breast",
      "Potato salad",
      "Mixed greens",
      "Lemon wedge",
      "Herb sauce"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      0.3,
      0.5
    ],
    "price_distribution": [
      60,
      15,
      10,
      5,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium",
    "storage_m3": [
      0.005,
      0.004,
      0.007,
      0.0005,
      0.0006
    ]
  },
  "1109": {
    "name": "Wild Mushroom Risotto",
    "ingredients": [
      "Arborio rice",
      "Mixed mushrooms",
      "White wine reduction",
      "Parmesan",
      "Truffle oil"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.3,
      0.5,
      0.05
    ],
    "price_distribution": [
      20,
      35,
      15,
      15,
      15
    ],
    "perishable": [
      false,
      true,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Executive",
    "storage_m3": [
      0.004,
      0.006,
      0.0004,
      0.0007,
      0.0001
    ]
  },
  "2290": {
    "name": "Beef Stroganoff",
    "ingredients": [
      "Tender beef strips",
      "Mushroom cream sauce",
      "Egg noodles",
      "Green beans",
      "Paprika garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      1.5,
      1.0,
      0.05
    ],
    "price_distribution": [
      65,
      15,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Executive",
    "storage_m3": [
      0.004,
      0.0015,
      0.003,
      0.005,
      0.0001
    ]
  },
  "1525": {
    "name": "Chicken Parmesan",
    "ingredients": [
      "Breaded chicken cutlets",
      "Marinara sauce",
      "Mozzarella",
      "Spaghetti",
      "Fresh basil"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      0.8,
      1.5,
      0.05
    ],
    "price_distribution": [
      55,
      10,
      15,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive",
    "storage_m3": [
      0.005,
      0.0015,
      0.001,
      0.003,
      0.0004
    ]
  },
  "2704": {
    "name": "Gourmet Vegetable Frittata",
    "ingredients": [
      "Free-range eggs",
      "Seasonal vegetables",
      "Artisanal cheese",
      "Fresh herbs",
      "Mixed greens salad"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.5,
      0.1,
      1.0
    ],
    "price_distribution": [
      40,
      20,
      25,
      5,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Executive",
    "storage_m3": [
      0.003,
      0.008,
      0.0007,
      0.0005,
      0.007
    ]
  },
  "1878": {
    "name": "Beef and Broccoli",
    "ingredients": [
      "Marinated beef strips",
      "Broccoli florets",
      "Oyster sauce",
      "Jasmine rice",
      "Toasted sesame garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      1.5,
      0.3,
      2.0,
      0.1
    ],
    "price_distribution": [
      65,
      15,
      5,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      false
    ],
    "type": "Meat",
    "price_tier": "Executive",
    "storage_m3": [
      0.004,
      0.009,
      0.0004,
      0.004,
      0.0001
    ]
  },
  "2640": {
    "name": "Tuna Nicoise Salad",
    "ingredients": [
      "Seared tuna",
      "Cooked eggs",
      "Green beans",
      "Baby potatoes",
      "Dijon vinaigrette"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      1.0,
      1.5,
      0.3
    ],
    "price_distribution": [
      60,
      10,
      10,
      10,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Executive",
    "storage_m3": [
      0.003,
      0.002,
      0.005,
      0.003,
      0.0004
    ]
  },
  "2577": {
    "name": "Chicken Noodle Soup",
    "ingredients": [
      "Roasted chicken",
      "Egg noodles",
      "Root vegetables",
      "Rich broth",
      "Fresh herbs"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      1.5,
      2.0,
      0.1
    ],
    "price_distribution": [
      50,
      15,
      15,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive",
    "storage_m3": [
      0.004,
      0.002,
      0.007,
      0.003,
      0.0005
    ]
  },
  "1754": {
    "name": "BBQ Chicken Platter",
    "ingredients": [
      "BBQ chicken",
      "House sauce",
      "Coleslaw",
      "Cornbread",
      "Pickled vegetables"
    ],
    "kg_per_10_persons": [
      2.5,
      0.5,
      1.0,
      1.0,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive",
    "storage_m3": [
      0.0075,
      0.0006,
      0.003,
      0.003,
      0.0008
    ]
  },
  "1971": {
    "name": "Gourmet Grain Bowl",
    "ingredients": [
      "Mixed beans",
      "Ancient grains",
      "House salsa",
      "Avocado",
      "Lime crema"
    ],
    "kg_per_10_persons": [
      1.0,
      1.5,
      0.5,
      1.0,
      0.3
    ],
    "price_distribution": [
      20,
      25,
      15,
      30,
      10
    ],
    "perishable": [
      false,
      false,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Executive",
    "storage_m3": [
      0.002,
      0.003,
      0.0006,
      0.003,
      0.0004
    ]
  },
  "2306": {
    "name": "Chicken and Rice Casserole",
    "ingredients": [
      "Free-range chicken",
      "Wild rice blend",
      "Creamy mushroom sauce",
      "Seasonal vegetables",
      "Herb crust"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      1.5,
      0.3
    ],
    "price_distribution": [
      55,
      15,
      15,
      10,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Executive",
    "storage_m3": [
      0.005,
      0.004,
      0.0015,
      0.008,
      0.0004
    ]
  },
  "2826": {
    "name": "Slow-Cooked Beef Ragu",
    "ingredients": [
      "Slow-simmered beef",
      "San Marzano tomato sauce",
      "Fresh pasta",
      "Aged Parmesan",
      "Fresh basil"
    ],
    "kg_per_10_persons": [
      2.5,
      1.5,
      2.0,
      0.5,
      0.1
    ],
    "price_distribution": [
      55,
      15,
      15,
      12,
      3
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Specialty",
    "storage_m3": [
      0.005,
      0.0025,
      0.004,
      0.0007,
      0.0005
    ]
  },
  "2139": {
    "name": "Mediterranean Vegetable Stew",
    "ingredients": [
      "Organic vegetables",
      "Imported olive oil",
      "Mediterranean herbs",
      "Pearl couscous",
      "Feta cheese"
    ],
    "kg_per_10_persons": [
      3.0,
      0.3,
      0.1,
      2.0,
      0.5
    ],
    "price_distribution": [
      45,
      20,
      10,
      15,
      10
    ],
    "perishable": [
      true,
      false,
      false,
      false,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Specialty",
    "storage_m3": [
      0.015,
      0.0003,
      0.0002,
      0.004,
      0.0007
    ]
  },
  "1230": {
    "name": "Teriyaki Salmon Bowl",
    "ingredients": [
      "Sustainable salmon",
      "House teriyaki glaze",
      "Sushi rice",
      "Asian vegetables",
      "Black sesame"
    ],
    "kg_per_10_persons": [
      2.0,
      0.5,
      2.0,
      1.0,
      0.05
    ],
    "price_distribution": [
      70,
      10,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      false,
      true,
      false
    ],
    "type": "Fish",
    "price_tier": "Specialty",
    "storage_m3": [
      0.004,
      0.0007,
      0.004,
      0.005,
      0.0001
    ]
  },
  "1207": {
    "name": "Authentic Carbonara",
    "ingredients": [
      "Fresh egg pasta",
      "Imported pancetta",
      "Farm-fresh eggs",
      "Pecorino Romano",
      "Cracked black pepper"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      1.0,
      0.5,
      0.02
    ],
    "price_distribution": [
      25,
      35,
      15,
      23,
      2
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Specialty",
    "storage_m3": [
      0.004,
      0.002,
      0.0015,
      0.0008,
      0.00005
    ]
  },
  "2322": {
    "name": "Gourmet Beef Enchiladas",
    "ingredients": [
      "Braised beef",
      "Handmade tortillas",
      "House enchilada sauce",
      "Mexican cheese blend",
      "Crema"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      1.0,
      0.8,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      15,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Specialty",
    "storage_m3": [
      0.004,
      0.0025,
      0.0015,
      0.001,
      0.0006
    ]
  },
  "2492": {
    "name": "Mediterranean Mezze Platter",
    "ingredients": [
      "Housemade falafel",
      "Artisanal hummus",
      "Freshly baked pita",
      "Greek salad",
      "Tzatziki sauce"
    ],
    "kg_per_10_persons": [
      1.5,
      1.0,
      1.0,
      2.0,
      0.8
    ],
    "price_distribution": [
      30,
      20,
      15,
      25,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.003,
      0.0015,
      0.003,
      0.01,
      0.001
    ]
  },
  "1216": {
    "name": "Gourmet Curry Selection",
    "ingredients": [
      "Premium vegetables",
      "House curry blend",
      "Saffron basmati rice",
      "Garlic naan",
      "Fresh herb garnish"
    ],
    "kg_per_10_persons": [
      2.5,
      0.3,
      2.0,
      1.0,
      0.1
    ],
    "price_distribution": [
      40,
      25,
      20,
      10,
      5
    ],
    "perishable": [
      true,
      false,
      false,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.012,
      0.0004,
      0.004,
      0.003,
      0.0005
    ]
  },
  "1727": {
    "name": "Scampi Linguine",
    "ingredients": [
      "Jumbo wild shrimp",
      "White wine garlic sauce",
      "Handmade linguine",
      "Cherry tomatoes",
      "Fresh herbs"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      0.8,
      0.1
    ],
    "price_distribution": [
      65,
      10,
      15,
      7,
      3
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.005,
      0.0015,
      0.004,
      0.002,
      0.0005
    ]
  },
  "1902": {
    "name": "Chef's Signature Pasta",
    "ingredients": [
      "Artisanal pasta",
      "San Marzano tomato sauce",
      "Fresh basil",
      "Imported Parmesan",
      "Extra virgin olive oil"
    ],
    "kg_per_10_persons": [
      2.5,
      1.5,
      0.2,
      0.5,
      0.2
    ],
    "price_distribution": [
      30,
      25,
      5,
      30,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.005,
      0.0025,
      0.001,
      0.0008,
      0.0002
    ]
  },
  "1247": {
    "name": "Gourmet Chicken Alfredo",
    "ingredients": [
      "Free-range chicken breast",
      "Fresh fettuccine",
      "House-made cream sauce",
      "Organic broccoli",
      "Aged Parmesan"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      1.0,
      0.3
    ],
    "price_distribution": [
      45,
      20,
      15,
      10,
      10
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.005,
      0.004,
      0.0015,
      0.006,
      0.0005
    ]
  },
  "2304": {
    "name": "Signature Pesto Pasta",
    "ingredients": [
      "Bronze-die pasta",
      "House basil pesto",
      "Heirloom tomatoes",
      "Toasted pine nuts",
      "24-month aged Parmesan"
    ],
    "kg_per_10_persons": [
      2.0,
      0.5,
      1.0,
      0.2,
      0.5
    ],
    "price_distribution": [
      25,
      20,
      15,
      15,
      25
    ],
    "perishable": [
      true,
      true,
      true,
      false,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.004,
      0.0007,
      0.004,
      0.0005,
      0.0008
    ]
  },
  "1543": {
    "name": "Prime Beef Tacos",
    "ingredients": [
      "Prime beef",
      "Hand-pressed tortillas",
      "Micro greens",
      "Aged cheddar",
      "House-made salsa"
    ],
    "kg_per_10_persons": [
      1.8,
      1.0,
      0.3,
      0.5,
      0.5
    ],
    "price_distribution": [
      60,
      10,
      10,
      12,
      8
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.004,
      0.0025,
      0.002,
      0.0008,
      0.0006
    ]
  },
  "1770": {
    "name": "Heritage Pork Stir Fry",
    "ingredients": [
      "Heritage pork tenderloin",
      "Seasonal organic vegetables",
      "Aged soy reduction",
      "Handmade noodles",
      "Rare spices"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      0.5,
      1.5,
      0.05
    ],
    "price_distribution": [
      60,
      15,
      10,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.004,
      0.01,
      0.0006,
      0.003,
      0.0001
    ]
  },
  "2126": {
    "name": "Deluxe Fish Curry",
    "ingredients": [
      "Wild-caught fish",
      "Coconut curry sauce",
      "Organic jasmine rice",
      "Exotic vegetables",
      "Micro herb garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      1.0,
      2.0,
      1.0,
      0.1
    ],
    "price_distribution": [
      65,
      15,
      10,
      8,
      2
    ],
    "perishable": [
      true,
      true,
      false,
      true,
      true
    ],
    "type": "Fish",
    "price_tier": "Premium Restaurant",
    "storage_m3": [
      0.004,
      0.0015,
      0.004,
      0.006,
      0.0005
    ]
  },
  "1558": {
    "name": "Artisanal Vegetable Lasagna",
    "ingredients": [
      "Handcrafted pasta sheets",
      "Seasonal organic vegetables",
      "San Marzano tomato sauce",
      "Artisanal béchamel",
      "Aged cheese blend"
    ],
    "kg_per_10_persons": [
      2.0,
      2.5,
      1.0,
      1.0,
      0.8
    ],
    "price_distribution": [
      25,
      20,
      20,
      15,
      20
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining",
    "storage_m3": [
      0.004,
      0.012,
      0.0015,
      0.0015,
      0.001
    ]
  },
  "2581": {
    "name": "Chef's Signature Soup",
    "ingredients": [
      "Heirloom lentils",
      "Seasonal organic vegetables",
      "House-made vegetable stock",
      "Imported herbs",
      "Artisanal sourdough"
    ],
    "kg_per_10_persons": [
      1.5,
      2.0,
      3.0,
      0.2,
      1.0
    ],
    "price_distribution": [
      25,
      30,
      15,
      15,
      15
    ],
    "perishable": [
      false,
      true,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining",
    "storage_m3": [
      0.003,
      0.01,
      0.004,
      0.0006,
      0.003
    ]
  },
  "1962": {
    "name": "Slow-Braised Chicken",
    "ingredients": [
      "Heritage chicken",
      "Organic root vegetables",
      "House-made bone broth",
      "Handcrafted dumplings",
      "Rare herb blend"
    ],
    "kg_per_10_persons": [
      3.0,
      2.0,
      2.0,
      1.0,
      0.1
    ],
    "price_distribution": [
      55,
      15,
      15,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      true
    ],
    "type": "Meat",
    "price_tier": "Fine Dining",
    "storage_m3": [
      0.009,
      0.01,
      0.003,
      0.002,
      0.0002
    ]
  },
  "2444": {
    "name": "Signature Sweet & Sour",
    "ingredients": [
      "Free-range chicken",
      "House-made sweet & sour sauce",
      "Organic vegetables",
      "Fresh pineapple",
      "Premium jasmine rice"
    ],
    "kg_per_10_persons": [
      2.5,
      1.0,
      1.5,
      1.0,
      2.0
    ],
    "price_distribution": [
      60,
      10,
      15,
      10,
      5
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Meat",
    "price_tier": "Fine Dining",
    "storage_m3": [
      0.006,
      0.0015,
      0.008,
      0.004,
      0.004
    ]
  },
  "2867": {
    "name": "Gourmet Stuffed Peppers",
    "ingredients": [
      "Heirloom bell peppers",
      "Ancient grain filling",
      "Imported tomato coulis",
      "Artisanal cheese",
      "Micro herb garnish"
    ],
    "kg_per_10_persons": [
      2.0,
      2.0,
      1.0,
      0.8,
      0.1
    ],
    "price_distribution": [
      30,
      25,
      15,
      25,
      5
    ],
    "perishable": [
      true,
      false,
      true,
      true,
      true
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining",
    "storage_m3": [
      0.012,
      0.004,
      0.0015,
      0.001,
      0.0005
    ]
  },
  "1445": {
    "name": "Chef's Quiche",
    "ingredients": [
      "All-butter pastry",
      "Organic free-range eggs",
      "Seasonal vegetables",
      "Imported cheese",
      "Truffle essence"
    ],
    "kg_per_10_persons": [
      1.5,
      2.0,
      1.5,
      0.8,
      0.05
    ],
    "price_distribution": [
      20,
      25,
      15,
      25,
      15
    ],
    "perishable": [
      true,
      true,
      true,
      true,
      false
    ],
    "type": "Vegetarian",
    "price_tier": "Fine Dining",
    "storage_m3": [
      0.003,
      0.003,
      0.008,
      0.001,
      0.0001
    ]
  }
}